In [0]:
# Try out MLFlow 3 preview
#%pip install mlflow --upgrade --pre
#%pip install mlflow databricks-agents --upgrade --pre

In [0]:
dbutils.library.restartPython()

In [0]:
import mlflow
import pandas as pd

from sklearn.linear_model import LinearRegression
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import root_mean_squared_error

mlflow.__version__

In [0]:
#%pip install databricks-sdk --upgrade

In [0]:
#%pip show databricks-sdk | grep -oP '(?<=Version: )\S+'

In [0]:
year = 2023
month = "03"

In [0]:
def read_dataframe(year, month):
    url = f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year}-{int(month):02d}.parquet'
    df = pd.read_parquet(url)

    return df

df_train = read_dataframe(year=year, month=month)
df_train  # HW Q3

In [0]:
def read_dataframe(year, month):
    url = f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year}-{int(month):02d}.parquet'
    df = pd.read_parquet(url)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']

    return df

year = 2023
month = "03"
df_train = read_dataframe(year=year, month=month)
df_train  # HW Q4

In [0]:
def create_X(df, dv=None):
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')

    if dv is None:
        dv = DictVectorizer(sparse=True)
        X = dv.fit_transform(dicts)
    else:
        X = dv.transform(dicts)

    return X, dv

X_train, dv = create_X(df_train)

target = 'duration'
y_train = df_train[target].values


In [0]:
import sklearn
sklearn.__version__

In [0]:
#mlflow.sklearn.autolog(disable=True)
mlflow.sklearn.autolog(log_datasets=False, log_model_signatures=False)
lr = LinearRegression()
lr.fit(X_train, y_train)

lr.intercept_  # HW Q5